# Lesson 5: Human in the Loop

**Lesson objective**: Get feedback on answers from a human operator

In this lab, you’ll learn how to make Workflows easy to iterate on answers to the questionnaire by getting feedback on answers from the human operator and re-answering when necessary.

<div style="background-color:#fff1d7; padding:15px;"> <b> Note</b>: Make sure to run the notebook cell by cell. Please try to avoid running all cells at once.</div>

In [1]:
import os, json
from llama_parse import LlamaParse
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import (
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context
)
from IPython.display import display, HTML
from helper import extract_html_content
from llama_index.utils.workflow import draw_all_possible_flows
from llama_index.core import SimpleDirectoryReader

In [2]:
import getpass
import os

INFERENCE_SERVER_URL = "http://localhost:8989"
# MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
# MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"

API_KEY= "alanliuxiang"

from llama_index.llms.openai_like import OpenAILike

model = OpenAILike(
  model=MODEL_NAME,
  api_key=API_KEY,
  api_base= f"{INFERENCE_SERVER_URL}/v1",
  context_window=1234,
  is_chat_model=True,  # supports chat completions
  is_function_calling_model=True # supports tools/functions in the api
)


In [3]:
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter

# embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
embed_model = HuggingFaceEmbedding()

Settings.llm = model
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 4096

In [4]:
import nest_asyncio
nest_asyncio.apply()

## Adding a feedback loop

Here's what you built in lesson 4:

<img width="400" src="images/L4.png">

LLMs are amazing, but they are best used to augment rather than replace a human. Your current form-filler does an excellent job figuring out what fields need to be filled in, and gets most of the fields right, but there are a couple where it needs a little help. To take care of those, you'll create a "human in the loop" workflow, where you can optionally provide feedback to the agent you've created and have it incorporated into the results.

This is what you'll implement in this notebook:

<img width="500" src="images/L5.png">

The changes you're going to make here are:
1. Use the `InputRequiredEvent` and `HumanResponseEvent`, new special events specifically designed to allow you to exit the workflow, and get feedback back into it.
2. You used to have a single step which parsed your form and fired off all your questions. Since we now might loop back and ask questions several times, we don't need to parse the form every time, so we'll split up those steps. This kind of refactoring is very common as you create a more complex workflow:
   - Your new `generate_questions` step will be triggered either by a `GenerateQuestionsEvent`, triggered by the form parser, or by a `FeedbackEvent`, which is the loop we'll take after getting feedback.
3. `fill_in_application` will emit an `InputRequiredEvent`, and in the `external_step` you'll wait for a `HumanResponseEvent`. This will pause the whole workflow waiting for outside input.
4. Finally, you'll use the LLM to parse the feedback and decide whether it means you should continue and output the results, or if you need to loop back.

In [5]:
# new!
from llama_index.core.workflow import InputRequiredEvent, HumanResponseEvent

In [6]:
class ParseFormEvent(Event):
    application_form: str

class QueryEvent(Event):
    query: str
    field: str
    
class ResponseEvent(Event):
    response: str

# new!
class FeedbackEvent(Event):
    feedback: str

class GenerateQuestionsEvent(Event):
    pass

In [7]:
class RAGWorkflow(Workflow):
    
    storage_dir = "./storage"
    llm: model
    query_engine: VectorStoreIndex

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        if not ev.application_form:
            raise ValueError("No application form provided")

        # define the LLM to work with
        self.llm = model

        # ingest the data and set up the query engine
        if os.path.exists(self.storage_dir):
            # you've already ingested the resume document
            storage_context = StorageContext.from_defaults(persist_dir=
                                                           self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            # parse and load the resume document
            documents = SimpleDirectoryReader(ev.resume_file).load_data()
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=embed_model
            )
            index.storage_context.persist(persist_dir=self.storage_dir)

        # create a query engine
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        # you no longer need a query to be passed in, 
        # you'll be generating the queries instead 
        # let's pass the application form to a new step to parse it
        return ParseFormEvent(application_form=ev.application_form)

    # new - separated the form parsing from the question generation
    @step
    async def parse_form(self, ctx: Context, ev: ParseFormEvent) -> GenerateQuestionsEvent:
        reader = SimpleDirectoryReader(
            input_files=[ev.application_form]
        )
        
        result = reader.load_data()

        raw_json = self.llm.complete(
            f"This is a parsed form. Convert it into a JSON object containing only the list of fields to be filled in, in the form {{ fields: [...] }}. <form>{result}</form>. Return JSON ONLY, no markdown.")
        fields = json.loads(raw_json.text)["fields"]

        await ctx.set("fields_to_fill", fields)

        return GenerateQuestionsEvent()

    # new - this step can get triggered either by GenerateQuestionsEvent or a FeedbackEvent
    @step
    async def generate_questions(self, ctx: Context, ev: GenerateQuestionsEvent | FeedbackEvent) -> QueryEvent:

        # get the list of fields to fill in
        fields = await ctx.get("fields_to_fill")

        # generate one query for each of the fields, and fire them off
        for field in fields:
            question = f"How would you answer this question about the candidate? <field>{field}</field>"
            ctx.send_event(QueryEvent(
                field=field,
                query=question
            ))

        # store the number of fields so we know how many to wait for later
        await ctx.set("total_fields", len(fields))
        return
        
    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> ResponseEvent:
        response = self.query_engine.query(f"This is a question about the specific resume we have in our database: {ev.query}")
        return ResponseEvent(field=ev.field, response=response.response)

  
    # new - we now emit an InputRequiredEvent
    @step
    async def fill_in_application(self, ctx: Context, ev: ResponseEvent) -> InputRequiredEvent:
        # get the total number of fields to wait for
        total_fields = await ctx.get("total_fields")

        responses = ctx.collect_events(ev, [ResponseEvent] * total_fields)
        if responses is None:
            return None # do nothing if there's nothing to do yet

        # we've got all the responses!
        responseList = "\n".join("Field: " + r.field + "\n" + "Response: " + r.response for r in responses)

        result = self.llm.complete(f"""
            You are given a list of fields in an application form and responses to
            questions about those fields from a resume. Combine the two into a list of
            fields and succinct, factual answers to fill in those fields.

            <responses>
            {responseList}
            </responses>
        """)

        # new! save the result for later
        await ctx.set("filled_form", str(result))

        # new! Let's get a human in the loop
        return InputRequiredEvent(
            prefix="How does this look? Give me any feedback you have on any of the answers.",
            result=result
        )

    # new! Accept the feedback.
    @step
    async def get_feedback(self, ctx: Context, ev: HumanResponseEvent) -> FeedbackEvent | StopEvent:

        result = self.llm.complete(f"""
            You have received some human feedback on the form-filling task you've done.
            Does everything look good, or is there more work to be done?
            <feedback>
            {ev.response}
            </feedback>
            If everything is fine, respond with just the word 'OKAY'.
            If there's any other feedback, respond with just the word 'FEEDBACK'.
        """)

        verdict = result.text.strip()
        # verdict = result.strip()

        print(f"LLM says the verdict was {verdict}")
        if (verdict == "OKAY"):
            return StopEvent(result=await ctx.get("filled_form"))
        else:
            return FeedbackEvent(feedback=ev.response)


Okay! Your workflow is now ready to get some feedback, but how do we actually get it? The `InputRequiredEvent` is an event in the event stream, just like the `ProgressEvents` and `TextEvents` you've seen in lesson 2. You can intercept it the same way you did those, and use the `send_event` method on the context to send back a `HumanResponseEvent`.

In [8]:
w = RAGWorkflow(timeout=600, verbose=False)
handler = w.run(
    resume_file="data/fake_resume.pdf",
    application_form="data/fake_application_form.pdf"
)

async for event in handler.stream_events():
    if isinstance(event, InputRequiredEvent):
        print("We've filled in your form! Here are the results:\n")
        print(event.result)
        # now ask for input from the keyboard
        response = input(event.prefix)
        handler.ctx.send_event(
            HumanResponseEvent(
                response=response
            )
        )

response = await handler
print("Agent complete! Here's your final result:")
print(str(response))

Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/index_store.json.


/tmp/ipykernel_8046/4163393584.py:56: DeprecationWarning: Context.set(key, value) is deprecated. Use 'await ctx.store.set(key, value)' instead.
  await ctx.set("fields_to_fill", fields)
/tmp/ipykernel_8046/4163393584.py:65: DeprecationWarning: Context.get() is deprecated. Use 'await ctx.store.get()' instead.
  fields = await ctx.get("fields_to_fill")
/tmp/ipykernel_8046/4163393584.py:76: DeprecationWarning: Context.set(key, value) is deprecated. Use 'await ctx.store.set(key, value)' instead.
  await ctx.set("total_fields", len(fields))
/tmp/ipykernel_8046/4163393584.py:89: DeprecationWarning: Context.get() is deprecated. Use 'await ctx.store.get()' instead.
  total_fields = await ctx.get("total_fields")
/tmp/ipykernel_8046/4163393584.py:109: DeprecationWarning: Context.set(key, value) is deprecated. Use 'await ctx.store.set(key, value)' instead.
  await ctx.set("filled_form", str(result))


We've filled in your form! Here are the results:

Field: First Name
Response: Sarah

Field: Last Name
Response: Chen

Field: Email
Response: sarah.chen@email.com

Field: Phone
Response: Not provided in the context.

Field: Linkedin
Response: linkedin.com/in/sarahcodes

Field: Project Portfolio
1. EcoTrack: A full-stack application built with React, Node.js, and MongoDB, designed to track carbon footprints with personalized sustainability recommendations. Featured in TechCrunch's "Top 10 Environmental Impact Apps of 2023."
2. ChatFlow: A real-time chat application using WebSocket protocol and React, featuring end-to-end encryption and message persistence, serving 5000+ monthly active users.

Field: Degree
Response: Bachelor of Science in Computer Science from the University of California, Berkeley, graduated in 2017.

Field: Graduation Date
Response: 2017

Field: Current Job title
Response: Junior Web Developer

Field: Current Employer
Response: TechFlow Solutions, San Francisco, CA

Fi

How does this look? Give me any feedback you have on any of the answers. great


LLM says the verdict was OKAY
Agent complete! Here's your final result:
Field: First Name
Response: Sarah

Field: Last Name
Response: Chen

Field: Email
Response: sarah.chen@email.com

Field: Phone
Response: Not provided in the context.

Field: Linkedin
Response: linkedin.com/in/sarahcodes

Field: Project Portfolio
1. EcoTrack: A full-stack application built with React, Node.js, and MongoDB, designed to track carbon footprints with personalized sustainability recommendations. Featured in TechCrunch's "Top 10 Environmental Impact Apps of 2023."
2. ChatFlow: A real-time chat application using WebSocket protocol and React, featuring end-to-end encryption and message persistence, serving 5000+ monthly active users.

Field: Degree
Response: Bachelor of Science in Computer Science from the University of California, Berkeley, graduated in 2017.

Field: Graduation Date
Response: 2017

Field: Current Job title
Response: Junior Web Developer

Field: Current Employer
Response: TechFlow Solutions,

/tmp/ipykernel_8046/4163393584.py:136: DeprecationWarning: Context.get() is deprecated. Use 'await ctx.store.get()' instead.
  return StopEvent(result=await ctx.get("filled_form"))


## Using the Feedback

Okay! Now let's further modify things to actually do something useful with the feedback in `generate_questions` step. This involves checking if there's feedback, and appending it to the questions. In this simple example, we're going to append the feedback to every question in case it's relevant, but a more sophisticated agent might apply it only to the fields where the feedback applied.

<img width="500" src="images/L5-use_feedback.png">

In [9]:
class RAGWorkflow(Workflow):
    
    storage_dir = "./storage"
    llm: model
    query_engine: VectorStoreIndex

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> ParseFormEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        if not ev.application_form:
            raise ValueError("No application form provided")

        # define the LLM to work with
        self.llm = model

        # ingest the data and set up the query engine
        if os.path.exists(self.storage_dir):
            # you've already ingested the resume document
            storage_context = StorageContext.from_defaults(persist_dir=
                                                           self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            # parse and load the resume document
            documents = SimpleDirectoryReader(ev.resume_file).load_data()
            # embed and index the documents
            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=embed_model
            )
            index.storage_context.persist(persist_dir=self.storage_dir)

        # create a query engine
        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)

        # let's pass the application form to a new step to parse it
        return ParseFormEvent(application_form=ev.application_form)

    # form parsing
    @step
    async def parse_form(self, ctx: Context, ev: ParseFormEvent) -> GenerateQuestionsEvent:
        reader = SimpleDirectoryReader(
            input_files=[ev.application_form]
        )
        
        result = reader.load_data()
        raw_json = self.llm.complete(
            f"This is a parsed form. Convert it into a JSON object containing only the list of fields to be filled in, in the form {{ fields: [...] }}. <form>{result}</form>. Return JSON ONLY, no markdown.")
        # fields = json.loads(raw_json)["fields"]
        fields = json.loads(raw_json.text)["fields"]

        await ctx.set("fields_to_fill", fields)

        return GenerateQuestionsEvent()

    # generate questions
    @step
    async def generate_questions(self, ctx: Context, ev: GenerateQuestionsEvent | FeedbackEvent) -> QueryEvent:

        # get the list of fields to fill in
        fields = await ctx.get("fields_to_fill")

        # generate one query for each of the fields, and fire them off
        for field in fields:
            question = f"How would you answer this question about the candidate? <field>{field}</field>"

            # new! Is there feedback? If so, add it to the query:
            if hasattr(ev,"feedback"):
                question += f"""
                    \nWe previously got feedback about how we answered the questions.
                    It might not be relevant to this particular field, but here it is:
                    <feedback>{ev.feedback}</feedback>
                """
            
            ctx.send_event(QueryEvent(
                field=field,
                query=question
            ))

        # store the number of fields so we know how many to wait for later
        await ctx.set("total_fields", len(fields))
        return
        
    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> ResponseEvent:
        response = self.query_engine.query(f"This is a question about the specific resume we have in our database: {ev.query}")
        return ResponseEvent(field=ev.field, response=response.response)

  
    # Get feedback from the human
    @step
    async def fill_in_application(self, ctx: Context, ev: ResponseEvent) -> InputRequiredEvent:
        # get the total number of fields to wait for
        total_fields = await ctx.get("total_fields")

        responses = ctx.collect_events(ev, [ResponseEvent] * total_fields)
        if responses is None:
            return None # do nothing if there's nothing to do yet

        # we've got all the responses!
        responseList = "\n".join("Field: " + r.field + "\n" + "Response: " + r.response for r in responses)

        result = self.llm.complete(f"""
            You are given a list of fields in an application form and responses to
            questions about those fields from a resume. Combine the two into a list of
            fields and succinct, factual answers to fill in those fields.

            <responses>
            {responseList}
            </responses>
        """)

        # save the result for later
        await ctx.set("filled_form", str(result))

        # Fire off the feedback request
        return InputRequiredEvent(
            prefix="How does this look? Give me any feedback you have on any of the answers.",
            result=result
        )

    # Accept the feedback when a HumanResponseEvent fires
    @step
    async def get_feedback(self, ctx: Context, ev: HumanResponseEvent) -> FeedbackEvent | StopEvent:

        result = self.llm.complete(f"""
            You have received some human feedback on the form-filling task you've done.
            Does everything look good, or is there more work to be done?
            <feedback>
            {ev.response}
            </feedback>
            If everything is fine, respond with just the word 'OKAY'.
            If there's any other feedback, respond with just the word 'FEEDBACK'.
        """)

        verdict = result.text.strip()

        print(f"LLM says the verdict was {verdict}")
        if (verdict == "OKAY"):
            return StopEvent(result=await ctx.get("filled_form"))
        else:
            return FeedbackEvent(feedback=ev.response)


Now run the workflow and give feedback.

In [10]:
w = RAGWorkflow(timeout=600, verbose=False)
handler = w.run(
    resume_file="data/fake_resume.pdf",
    application_form="data/fake_application_form.pdf"
)

async for event in handler.stream_events():
    if isinstance(event, InputRequiredEvent):
        print("We've filled in your form! Here are the results:\n")
        print(event.result)
        # now ask for input from the keyboard
        response = input(event.prefix)
        handler.ctx.send_event(
            HumanResponseEvent(
                response=response
            )
        )

response = await handler
print("Agent complete! Here's your final result:")
print(str(response))

Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/index_store.json.


/tmp/ipykernel_8046/1319504359.py:54: DeprecationWarning: Context.set(key, value) is deprecated. Use 'await ctx.store.set(key, value)' instead.
  await ctx.set("fields_to_fill", fields)
/tmp/ipykernel_8046/1319504359.py:63: DeprecationWarning: Context.get() is deprecated. Use 'await ctx.store.get()' instead.
  fields = await ctx.get("fields_to_fill")
/tmp/ipykernel_8046/1319504359.py:83: DeprecationWarning: Context.set(key, value) is deprecated. Use 'await ctx.store.set(key, value)' instead.
  await ctx.set("total_fields", len(fields))
/tmp/ipykernel_8046/1319504359.py:96: DeprecationWarning: Context.get() is deprecated. Use 'await ctx.store.get()' instead.
  total_fields = await ctx.get("total_fields")
/tmp/ipykernel_8046/1319504359.py:116: DeprecationWarning: Context.set(key, value) is deprecated. Use 'await ctx.store.set(key, value)' instead.
  await ctx.set("filled_form", str(result))


We've filled in your form! Here are the results:

Field: First Name
Response: Sarah

Field: Last Name
Response: Chen

Field: Email
Response: sarah.chen@email.com

Field: Linkedin
Response: linkedin.com/in/sarahcodes

Field: Project Portfolio
Response: The candidate, Sarah Chen, has developed applications such as EcoTrack (React, Node.js, MongoDB) and ChatFlow (WebSocket, React). EcoTrack was featured in TechCrunch's "Top 10 Environmental Impact Apps of 2023". She has experience in microservices architecture, having architected and implemented a platform serving 100K+ daily users at TechFlow Solutions.

Field: Degree
Response: Bachelor of Science in Computer Science from the University of California, Berkeley, graduated in 2017.

Field: Graduation Date
Response: Around the end of 2017, assuming a four-year program.

Field: Current Job title
Response: Senior Full Stack Developer

Field: Current Employer
Response: TechFlow Solutions, San Francisco, CA

Field: Technical Skills
Response: Fr

How does this look? Give me any feedback you have on any of the answers. okay


LLM says the verdict was FEEDBACK
We've filled in your form! Here are the results:

Field: First Name
Response: Sarah

Field: Last Name
Response: Chen

Field: Email
Response: LinkedIn profile is linkedin.com/in/sarahchen.

Field: Phone
Response: No direct information available.

Field: Linkedin
Response: linkedin.com/in/sarahchen

Field: Project Portfolio
Response: The candidate has developed applications such as EcoTrack (React, Node.js, MongoDB) and ChatFlow (WebSocket protocol, React), showcasing skills in full-stack web development. EcoTrack featured in TechCrunch's "Top 10 Environmental Impact Apps of 2023."

Field: Degree
Response: Bachelor of Science in Computer Science from the University of California, Berkeley, with a minor in User Experience Design. Graduated in 2017.

Field: Graduation Date
Response: Likely graduated in 2017.

Field: Current Job title
Response: Senior Full Stack Developer

Field: Current Employer
Response: TechFlow Solutions, San Francisco, CA

Field: Techn

How does this look? Give me any feedback you have on any of the answers. good


LLM says the verdict was OKAY
Agent complete! Here's your final result:
Field: First Name
Response: Sarah

Field: Last Name
Response: Chen

Field: Email
Response: LinkedIn profile is linkedin.com/in/sarahchen.

Field: Phone
Response: No direct information available.

Field: Linkedin
Response: linkedin.com/in/sarahchen

Field: Project Portfolio
Response: The candidate has developed applications such as EcoTrack (React, Node.js, MongoDB) and ChatFlow (WebSocket protocol, React), showcasing skills in full-stack web development. EcoTrack featured in TechCrunch's "Top 10 Environmental Impact Apps of 2023."

Field: Degree
Response: Bachelor of Science in Computer Science from the University of California, Berkeley, with a minor in User Experience Design. Graduated in 2017.

Field: Graduation Date
Response: Likely graduated in 2017.

Field: Current Job title
Response: Senior Full Stack Developer

Field: Current Employer
Response: TechFlow Solutions, San Francisco, CA

Field: Technical Skills


/tmp/ipykernel_8046/1319504359.py:142: DeprecationWarning: Context.get() is deprecated. Use 'await ctx.store.get()' instead.
  return StopEvent(result=await ctx.get("filled_form"))


## Workflow Visualization

You can visualize the workflow you just created.

In [11]:
WORKFLOW_FILE = "workflows/feedback_workflow.html"
draw_all_possible_flows(w, filename=WORKFLOW_FILE)
html_content = extract_html_content(WORKFLOW_FILE)
display(HTML(html_content), metadata=dict(isolated=True))

workflows/feedback_workflow.html


## Success!

The agent now responds to Human-in-the-loop feedback and produces more accurate filled forms as a result. Congratulations!